In [1]:
########################################################################
##
##  Web Scraping Homework - Mission to Mars
##
##       Author: George Alonzo
##     Due Date: November 20, 2021
##
########################################################################

In [2]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, jsonify

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Tower\.wdm\drivers\chromedriver\win32\96.0.4664.45]


# NASA Mars News

In [4]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')

In [7]:
# loop over results to get article data

# NOT GOING TO ATTEMPT TRYING TO CLICK THE 'MORE' BUTTON AS IT IS NOT ENABLED ON THE PAGE
news_title=[]
paragraph_text=[]
for result in results:
    # scrape the article header 
    news_title.append(result.find('div', class_='content_title').text)

    # scrape the article subheader
    paragraph_text.append(result.find('div', class_='article_teaser_body').text)
    
    # print data
    print('-----------------')
    print(news_title)
    print(paragraph_text)

print("="*50)
print(f"Number of Headers scraped & stored: {len(news_title)}")
print(f"Number of Sub-Headers scraped & stored: {len(paragraph_text)}")

-----------------
['New Selfie Shows Curiosity, the Mars Chemist']
['The NASA rover performed a special chemistry experiment at the location captured in its newest self-portrait.']
-----------------
['New Selfie Shows Curiosity, the Mars Chemist', "NASA's Perseverance Rover Mission Getting in Shape for Launch"]
['The NASA rover performed a special chemistry experiment at the location captured in its newest self-portrait.', 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. ']
-----------------
['New Selfie Shows Curiosity, the Mars Chemist', "NASA's Perseverance Rover Mission Getting in Shape for Launch", "NASA's Mars 2020 Rover Goes Coast-to-Coast to Prep for Launch"]
['The NASA rover performed a special chemistry experiment at the location captured in its newest self-portrait.', 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red P

# JPL Mars Space Images - Featured Image

In [8]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [9]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [10]:
results = soup.find_all('div', class_='header')

In [11]:
for result in results:
    featured_image = result.find('img', class_='headerimage fade-in')['src']
    featured_image_url = "https://spaceimages-mars.com/"+featured_image
      
print(f"Header Image URL: {featured_image_url}")

Header Image URL: https://spaceimages-mars.com/image/featured/mars1.jpg


# Mars Facts

In [12]:
url = 'https://galaxyfacts-mars.com/'

In [13]:
tables = pd.read_html(url)
html_table = tables[0]
html_table = html_table.rename(columns={0:"Description",1:"Mars",2:"Earth"})
html_table = html_table.set_index("Description")
html_table = html_table.iloc[1:,:]
html_table

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


df=tables[0]
#Rename column headings
df.rename(columns={0:"Mars - Earth Comparison",1:"Mars",2:"Earth"},inplace=True)
#Drop first row since the column headings were imported as a row
df=df.iloc[1:,:]
df

In [14]:
html_table = html_table.to_html()

In [15]:
#Strip newlines
html_table = html_table.replace("\n", "")
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [16]:
url = 'https://marshemispheres.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [17]:
results = soup.find_all('div', class_='item')

In [18]:
sub_urls=[]
hemisphere_image_list =[]
for result in results:
    # look for clickable links
    href = result.find('a')['href']
    LinkName = result.find('h3').text
    sub_urls.append(url+href)
    
    browser.click_link_by_partial_text(LinkName)
    clicked_html = url+browser.html
    clicked_soup = BeautifulSoup(clicked_html, 'html.parser')
    clicked_results = clicked_soup.find_all('div', class_='cover')
    for clicked_result in clicked_results:
        hi_href = url+clicked_result.find('a')['href']
        hemisphere_image_dict={}
        hemisphere_image_dict["title"] = LinkName.rsplit(' ', 1)[0]
        hemisphere_image_dict["img_url"] = hi_href
        hemisphere_image_list.append(hemisphere_image_dict)
    browser.click_link_by_partial_text('Back')
    
print(f"Number of clickable links found: {len(sub_urls)}")
print("="*50)
hemisphere_image_list

C:\Users\Tower\anaconda3\envs\PythonData38\lib\site-packages\splinter\driver\webdriver\__init__.py:519: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


Number of clickable links found: 4


[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]

In [19]:
browser.quit()